# Extracting Vital Statistics from the CIA World Factbook with SQLite
## Executive Summary
In this project, I utilized SQLite to extract demographic and geographic data from the CIA World Factbook. I identified China as the most populous locale tracked by the CIA, and the Pitcairn Islands as the least populous. Malawi had the highest rate of population growth, and Bulgaria had the highest rate of population contraction (both excluding immigration and emigration). Greenland had the lowest population density; Macau, the largest. Finally, Puerto Rico had the largest water-to-land-area ratio.

## Introduction
The **['CIA World Factbook'](https://www.cia.gov/the-world-factbook/)** is a datasource that provides vital statistics of 266 world 'locales'. Locales can be:
* Countries
* Regions
* Oceans
* Unoccupied Islands

In this project, I use SQLite to work with a ***small subset*** of the CIA factbook to identify the maximum and minimum values of:
* Population
* Population Growth
* Population Density
* Land-to-Water Ratio

## Setup Jupyter to use SQLite

In [1]:
%%capture
%load_ext sql
%sql sqlite:///DataSets/CIA_Factbook/factbook.db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type = 'table';

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


## Data Format
As mentioned above, I will be working with a small subset of complete CIA World Factbook. Definitions for the columns in the factbook can be found [here](https://www.cia.gov/the-world-factbook/references/definitions-and-notes/).

In this data subset, the following columns are used:
1. code = a unique 2 letter locale code.
2. name = the name of the locale.
3. area = the total area of the locale in square km.
4. area_land = the land area of the locale in square km.
5. area_water = the area of sovreign water occupied by the locale in square km.
6. population = the total population of the locale.
7. population_growth = population growth rate in percent.
8. birth_rate = the number of births per 1,000 persons.
9. death_rate = the number of deaths per 1,000 persons.
10. migration_rate = the net migration rate per 1,000 persons. In this dataset, I am unable to determine if there is a correct sign on the migration rate, since it is not possible for all countries to have a migration rate >= 0.

## Analysis

### Population
A first glance at the maximum and minimum populations listed in the factbook show that the maximum population is over 7 billion people, and the miminum population is 0.

In [3]:
%%sql
SELECT MIN(population) AS 'Lowest Population',
       MAX(population) AS 'Highest Population',
       MIN(population_growth) AS 'Fastest Growth',
       MAX(population_growth) AS 'Slowest Growth'
  FROM facts

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Lowest Population,Highest Population,Fastest Growth,Slowest Growth
0,7256490011,0.0,4.02


Further examination shows that this region with the maximum population is 'The World', while the region(s) with the minimum populations are unoccupied islands and Antarctica.

In [4]:
%%sql
SELECT name,
       population
  FROM facts
 ORDER BY population
 LIMIT 20;

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


name,population
Ashmore and Cartier Islands,None
Coral Sea Islands,None
Heard Island and McDonald Islands,None
Clipperton Island,None
French Southern and Antarctic Lands,None
Bouvet Island,None
Jan Mayen,None
British Indian Ocean Territory,None
South Georgia and South Sandwich Islands,None
Navassa Island,None


When we exclude these non-country maximum and minimum locales, we can find the largest and smallest countries in the world.

In [5]:
%%sql
SELECT name AS 'Most Populous Locale',
       MAX(population) AS 'Population'
  FROM facts
 WHERE population < (SELECT MAX(population)
                                FROM facts)
 LIMIT 1;

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Most Populous Locale,Population
China,1367485388


In [6]:
%%sql
SELECT name AS 'Least Populous Locale',
       MIN(population) AS 'Population'
  FROM facts
 WHERE population > 0
 LIMIT 1;

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Least Populous Locale,Population
Pitcairn Islands,48


### Population Growth
I define the population growth rate as the birth rate less the death rate in each locale. Due to the unknown sign of the migration rate within the dataset, I will ignore its contribution to population growth (or decline)

In [7]:
%%sql
SELECT name AS 'Fastest Growing Locale',
       ROUND(MAX(birth_rate - death_rate),2) AS 'Net population change per 1,000 persons'
  FROM facts

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Fastest Growing Locale,"Net population change per 1,000 persons"
Malawi,33.15


In [8]:
%%sql
SELECT name AS 'Slowest Growing Locale',
       ROUND(MIN(birth_rate - death_rate),2) AS 'Net population change per 1,000 persons'
  FROM facts

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Slowest Growing Locale,"Net population change per 1,000 persons"
Bulgaria,-5.52


### Population Density
I define this as the reported locale population divided by the **land** area of the locale (different calculations choose to [include or exclude](https://en.wikipedia.org/wiki/Population_density#Human_densities) water area)

In [9]:
%%sql
SELECT name AS 'Highest population density',
       area_land AS 'Land Area (sq km)',
       population AS 'Population',
       ROUND(MAX(CAST(population AS FLOAT) / area_land),2) AS 'Persons per square km'
  FROM facts

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Highest population density,Land Area (sq km),Population,Persons per square km
Macau,28,592731,21168.96


In [10]:
%%sql
SELECT name AS 'Lowest population density',
       area_land AS 'Land Area (sq km)',
       population AS 'Population',
       ROUND((MIN(CAST(population AS FLOAT) / area_land)),2) AS 'Persons per square km'
  FROM facts
 WHERE (CAST(population AS FLOAT) / area_land) > 0
 ORDER BY (CAST(population AS FLOAT) / area_land)

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Lowest population density,Land Area (sq km),Population,Persons per square km
Greenland,2166086,57733,0.03


### Water to Land Ratio
The ratio of water to land for a given locale can vary from 0 (a landlocked locale) to near infinite (a tiny island).

An initial sampling of all locales with a water area of zero reveals that many island nations are listed as having a water area of zero. My assumption is that the water area reported in the Factbook is the sum of [Territorial Waters](https://en.wikipedia.org/wiki/Territorial_waters) and [Internal Waters](https://en.wikipedia.org/wiki/Internal_waters). Generally, *territorial waters* extend 12 nautical miles beyond a country's coast, so island nations should have roughly:

$$a_{water} = 44\pi(11+r_{land})$$

Where all variables are given in sq km, and an island is assumed to be circular.

In [11]:
%%sql
SELECT name AS 'Locale',
       area_land AS 'Land Area (sq km)',
       area_water AS 'Water Area (sq km)'
  FROM facts
 WHERE area_water = 0

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Locale,Land Area (sq km),Water Area (sq km)
Afghanistan,652230,0
Algeria,2381741,0
Andorra,468,0
Angola,1246700,0
Antigua and Barbuda,442,0
Bahrain,760,0
Barbados,430,0
Bhutan,38394,0
Cabo Verde,4033,0
Central African Republic,622984,0


To circumvent this issue, I attempted to exclude these mislabeled small islands from the calculation by removing locales with area_water = 0 and area_land < 1000 sq km.

**This filtering process also removed any truly landlocked countries from analysis.** The remaining entries in the list were countries with a land area > 1,000 sq km which had a non-zero water area.

In [12]:
%%sql
SELECT name AS 'Locale',
       area_land AS 'Land Area (sq km)',
       area_water AS 'Water Area (sq km)',
       ROUND((CAST(area_water AS FLOAT) / area_land),5) AS 'Water-Land Ratio'
  FROM facts
 WHERE (ROUND((CAST(area_water AS FLOAT) / area_land),5) != 0)
       AND (area_land > 1000)
 ORDER BY ROUND((CAST(area_water AS FLOAT) / area_land),5) DESC
 LIMIT 5

 * sqlite:///DataSets/CIA_Factbook/factbook.db
Done.


Locale,Land Area (sq km),Water Area (sq km),Water-Land Ratio
Puerto Rico,8870,4921,0.55479
"Bahamas, The",10010,3870,0.38661
Guinea-Bissau,28120,8005,0.28467
Malawi,94080,24404,0.2594
Netherlands,33893,7650,0.22571


## Conclusions
In this project, I utilized elementary SQL queries to extract population and land-area data from the CIA World Factbook. I identified the following:
* Population
 * **China** is the world's **largest country by population**
 * **The Pitcairn Islands** is the **smallest locale (by population)** tracked in the CIA Factbook.
* Population Growth
 * **Malawi** has the **highest rate of population growth** in the world (excluding immigration and emigration)
 * **Bulgaria** has the **lowest rate of population growth** in the world (excluding immmigration and emigration)
* Population Density
 * **Greenland** has the world's **lowest population density**
 * **Macau** has the world's **highest population density**
* **Puerto Rico** has the world's highest water-to-land ratio.